In [773]:
import pandas as pd
import numpy as np
import re
import datetime
import os
import dateparser

In [774]:
folder = r'D:\FTP-DATA\Analytics\Promo_rocket\23052021'
os.chdir(folder)

handbook = {'с': 'БанкиРУ',
           'в': 'РБК',
           'л': 'КЦ',
           'р': 'Не используем'}

In [775]:
writer = pd.ExcelWriter('Отчет_очереди_данные.xlsx')

In [776]:
# Обработаем файл с промокодами и номерами телефонов оттуда

queue_priority = [file for file in os.listdir() if all(['приоритет' in file.lower(), 'очередь' in file.lower(), '$' not in file])][0]

queue_priority = pd.read_excel(queue_priority, header=None)

queue_priority = queue_priority.dropna()

queue_priority = queue_priority[0].to_list()

queue_priority_indexes = [i for i, j in enumerate(queue_priority) if any([x.isalpha() for x in str(j).strip()])] + [len(queue_priority) - 1]

queue_priority_dct = {'date': [], 'Выдано промокодов КЦ': []}

for i, j in enumerate(queue_priority_indexes[:-1]):
    queue_priority_dct['date'].append(dateparser.parse(queue_priority[j].strip()).strftime('%d.%m.%Y'))
    queue_priority_dct['Выдано промокодов КЦ'].append(queue_priority_indexes[i + 1] - j - 1)
    
queue_priority_df = pd.DataFrame(queue_priority_dct)

first_date = datetime.datetime.strptime(queue_priority_df.date[0], '%d.%m.%Y')





In [777]:
queue_priority_df

date  Выдано промокодов КЦ
0   13.05.2021                   120
1   14.05.2021                    60
2   15.05.2021                    61
3   16.05.2021                    65
4   17.05.2021                   102
5   18.05.2021                    56
6   19.05.2021                    65
7   20.05.2021                    79
8   21.05.2021                    54
9   22.05.2021                    59
10  23.05.2021                    47
11  24.05.2021                   113
12  25.05.2021                    75

In [778]:
# Обработаем файл с сс1

cc_report = [file for file in os.listdir() if all(['cc' in file.lower(), 'rep' in file.lower(), '$' not in file])][0]

cc_report = pd.read_excel(cc_report, skiprows=2, usecols='A, P')

cc_report = cc_report.rename(columns={'Unnamed: 0': 'Дата', 'ВОО (2)': 'Кнопка ВОО (2)'})

cc_report = cc_report.iloc[cc_report['Дата'].to_list().index(first_date): -1, :]

cc_report['Кнопка ВОО (2)'] = cc_report['Кнопка ВОО (2)'].astype(int)

cc_report['Дата'] = cc_report['Дата'].dt.strftime('%d.%m.%Y')

cc_report = cc_report.merge(queue_priority_df, left_on='Дата', right_on='date')

In [779]:
# Обработаем данные электронной очереди

electro_queue = [file for file in os.listdir() if all(['лидер' in file.lower(), '$' not in file])][0]

electro_queue = pd.read_excel(electro_queue, skipfooter=1)

electro_queue = electro_queue[[col for col in electro_queue.columns if 'общий' not in str(col).lower()]]

electro_queue.columns = ['Названия строк'] + [col.strftime('%d.%m.%Y') for col in electro_queue.columns[1:]]

electro_queue = electro_queue.append(electro_queue.sum(axis=0), ignore_index=True)

for_rename = electro_queue['Названия строк'].iloc[-1]

electro_queue.iloc[-1, 0] = 'Использовано в э/очереди (все промокоды)'

electro_queue = electro_queue.iloc[-1, :]

electro_queue = pd.DataFrame({electro_queue.index[0]: electro_queue.index[1:], electro_queue.values[0]: electro_queue.values[1:]})

cc_report = cc_report.merge(electro_queue, left_on='Дата', right_on='Названия строк')

cc_report = cc_report.drop('Названия строк', axis=1)

cc_report = cc_report.T

cc_report.columns = cc_report.iloc[0, :]

cc_report = cc_report.drop(['date', 'Дата'])


## Делим промокоды по виду

In [780]:
promo_views = pd.DataFrame()

path = r'D:\FTP-DATA\Analytics\Promo_rocket\23052021\promo_days'

for file in os.listdir(path):
    if 'xlsx' in file and '$' not in file:
        date_name = file[:file.rfind('.')]
        temp_df = pd.read_excel(path + '\\' + file, usecols=['Отделение', 'Номер чека'])
        temp_df['Дата'] = datetime.datetime.strptime(date_name, '%d.%m.%Y')
        promo_views = promo_views.append(temp_df)

promo_views['Источник'] = promo_views['Номер чека'].apply(lambda x: handbook[x[0].lower()])

promo_views = promo_views.sort_values(by=['Источник', 'Отделение', 'Дата'])

call_center = promo_views[promo_views['Источник'] == 'КЦ'].copy()

bankiru = promo_views[promo_views['Источник'] == 'БанкиРУ'].copy()

rbk = promo_views[promo_views['Источник'] == 'РБК'].copy()

promo_views = promo_views.pivot_table(index='Отделение', columns='Дата', values='Источник', aggfunc='count', margins=True, margins_name='Общий итог')

call_center = call_center.pivot_table(index='Отделение', columns='Дата', values='Источник', aggfunc='count', margins=True, margins_name='Общий итог')

bankiru = bankiru.pivot_table(index='Отделение', columns='Дата', values='Источник', aggfunc='count', margins=True, margins_name='Общий итог')

rbk = rbk.pivot_table(index='Отделение', columns='Дата', values='Источник', aggfunc='count', margins=True, margins_name='Общий итог')

promo_views.columns = [col.strftime('%d.%m.%Y') for col in promo_views.columns[:-1]] + ['Общий итог']

call_center.columns = [col.strftime('%d.%m.%Y') for col in call_center.columns[:-1]] + ['Общий итог']

bankiru.columns = [col.strftime('%d.%m.%Y') for col in bankiru.columns[:-1]] + ['Общий итог']

rbk.columns = [col.strftime('%d.%m.%Y') for col in rbk.columns[:-1]] + ['Общий итог']


In [781]:
cc_report = cc_report.append(call_center.iloc[-1, :-1])

cc_report = cc_report.append(bankiru.iloc[-1, :-1])

cc_report = cc_report.append(rbk.iloc[-1, :-1])

cc_report.index = list(cc_report.index)[:-3] + ['Использовано в э/очереди (промокоды КЦ)', 'Использовано в э/очереди (промокоды БанкиРУ)', 'Использовано в э/очереди (промокоды РБК)']

## Подтянем БанкиРу

In [782]:
bankiru_visitors = [file for file in os.listdir() if all(['bankiru' in file.lower(), 'visitors' in file.lower(), '$' not in file])][0]

bankiru_visitors_spb = pd.read_excel(bankiru_visitors, sheet_name='Санкт-Петербург')

bankiru_visitors_msk = pd.read_excel(bankiru_visitors, sheet_name='Москва')

spb_index = bankiru_visitors_spb[bankiru_visitors_spb['Unnamed: 1'] == 'Обменный пункт'].index[1]

msk_index = bankiru_visitors_msk[bankiru_visitors_msk['Unnamed: 1'] == 'Обменный пункт'].index[1]

bankiru_visitors_spb = bankiru_visitors_spb.iloc[spb_index:, 1:]

bankiru_visitors_msk = bankiru_visitors_msk.iloc[msk_index:, 1:]

In [783]:
bankiru_visitors_spb.columns = bankiru_visitors_spb.iloc[0]

bankiru_visitors_spb.columns = ['Обменный пункт'] + [datetime.date(2021, *[int(i) for i in str(col).strip().split('.')[::-1]]) for col in bankiru_visitors_spb.columns[1:]]

bankiru_visitors_spb = bankiru_visitors_spb[[col for col in bankiru_visitors_spb.columns if any([col == 'Обменный пункт', col in list(pd.date_range(start=datetime.date(2021, 5, 21), end=datetime.datetime.today()))])]]

bankiru_visitors_spb = bankiru_visitors_spb.iloc[-1, :]

bankiru_visitors_msk.columns = bankiru_visitors_msk.iloc[0]

bankiru_visitors_msk.columns = ['Обменный пункт'] + [datetime.date(2021, *[int(i) for i in str(col).strip().split('.')[::-1]]) for col in bankiru_visitors_msk.columns[1:]]

bankiru_visitors_msk = bankiru_visitors_msk[[col for col in bankiru_visitors_msk.columns if any([col == 'Обменный пункт', col in list(pd.date_range(start=datetime.date(2021, 5, 21), end=datetime.datetime.today()))])]]

bankiru_visitors_msk = bankiru_visitors_msk.iloc[-1, :]

bankiru_visitors = bankiru_visitors_spb + bankiru_visitors_msk

bankiru_visitors = pd.DataFrame(bankiru_visitors).T

bankiru_visitors.iloc[0, 0] = 'Все, БанкиРУ'

bankiru_visitors.columns = ['Обменный пункт'] + [col.strftime('%d.%m.%Y') for col in bankiru_visitors.columns[1:]]

bankiru_series = pd.Series(bankiru.iloc[-1, :-1], name='Обменный пункт')

bankiru_visitors = bankiru_visitors.append(bankiru_series)

In [784]:
cc_report.to_excel(writer, sheet_name='Общее_таблица')

promo_views.to_excel(writer, sheet_name='Общее_отделения')

call_center_report = cc_report.loc[['Кнопка ВОО (2)', 'Выдано промокодов КЦ', 'Использовано в э/очереди (промокоды КЦ)'], :]

call_center_report = call_center_report.append(pd.Series(call_center_report.iloc[1, :] / call_center_report.iloc[0, :], name='Конверсия кнопка 2 --> выдано промокодов КЦ'))

call_center_report = call_center_report.append(pd.Series(call_center_report.iloc[2, :] / call_center_report.iloc[0, :], name='Конверсия кнопка 2 --> Использовано в э/очереди'))

call_center_report.iloc[2:4] = np.flip(call_center_report.to_numpy()[2:4], axis=0)

call_center_report.index = ['Кнопка ВОО (2)', 'Выдано промокодов КЦ',
       'Конверсия кнопка 2 --> выдано промокодов КЦ',
        'Использовано в э/очереди (промокоды КЦ)',
       'Конверсия кнопка 2 --> Использовано в э/очереди']

# call_center_report.iloc[2] = call_center_report.iloc[2].apply(lambda x: str(round(x * 100, 2)) + '%') форматированием в excel надо поменять
# call_center_report.iloc[4] = call_center_report.iloc[4].apply(lambda x: str(round(x * 100, 2)) + '%')

call_center_report.to_excel(writer, sheet_name='КЦ_таблица')

call_center.to_excel(writer, sheet_name='КЦ_отделения')

bankiru_visitors.to_excel(writer, sheet_name='БанкиРУ_таблица')

bankiru.to_excel(writer, sheet_name='БанкиРУ_отделения')

rbk.to_excel(writer, sheet_name='РБК_отделения')




In [785]:
writer.save()